# Colorization of Black and White Photos Using Neural Networks (Keras Implementation)


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from IPython.display import display, Image
from matplotlib.pyplot import imshow
from keras.layers import Conv2D, UpSampling2D, InputLayer
from keras.models import Sequential
from keras.preprocessing.image import img_to_array, load_img
from skimage.color import lab2rgb, rgb2lab

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/rural_and_urban_photos/train/urban"))



# Any results you write to the current directory are saved as output.

INPUT_IMAGE_SRC = '../input/rural_and_urban_photos/train/urban/urban_11.jpeg'

In [ ]:
display(Image(INPUT_IMAGE_SRC, width=225))

> The Lab color space describes mathematically all perceivable colors in the three dimensions **L for lightness** and **a** and **b** for the color components **green–red** and **blue–yellow**.

and the reason why we want to use it instead of RGB

> Unlike the RGB and CMYK color models, Lab color is **designed to approximate human vision**. It aspires to perceptual uniformity, and its L component closely matches human perception of lightness, although it does not take the Helmholtz–Kohlrausch effect into account. Thus, it can be used to make accurate color balance corrections by modifying output curves in the a and b components, or to adjust the lightness contrast using the L component. In RGB or CMYK spaces, which **model the output of physical devices rather than human visual perception**, these transformations can be done only with the help of appropriate blend modes in the editing application.

via https://en.wikipedia.org/wiki/Lab_color_space

Let's load the image and turn its **RBG** representation into **Lab**. Before doing that, we should normalize array represenation of the image. RGB values range from (0, 255), but for our purposes, it is useful to constraint them to the (0,1) range. We can do so, by diving the R, G, and B values of each pixes by the maximum (255). Having done that, we can proceed covenrting the RGB image to Lab. The resulting numpy array has a shape of `(width, height, 3)`, the 3 standing for the three different layers we end up with. The first layer is our Lightness component, or put simply the black and white representation we would use as inputs  in our training. The rest of the layers represent the respective color mappings and will be used as our training outputs.

In [ ]:
image = img_to_array(load_img(INPUT_IMAGE_SRC, target_size=(200,200))) / 255
lab_image = rgb2lab(image)
lab_image.shape

One more normalization step is needed, before we can extract the data. The color values  in the different Lab layers are in various ranges, as follows:
* L: (0, 100)
* a & b: (-128, 128)
via http://www.colourphil.co.uk/lab_lch_colour_space.shtml

For our purposes, we would want them all to be normalized and in the 



In [ ]:
lab_image_norm = (lab_image + [0, 128, 128]) / [100, 255, 255]

Having turned the image into an Lab represantion, let's define the inputs and outputs. The input in our case will be the black and white layer (`lab_image[:,:,0]`)

In [ ]:
# The input will be the black and white layer
X = lab_image_norm[:,:,0]
# The outpts will be the ab channels
Y = lab_image_norm[:,:,1:]

# The Conv2D layer we will use later expects the inputs and training outputs to be of the following format:
# (samples, rows, cols, channels), so we need to do some reshaping
# https://keras.io/layers/convolutional/
X = X.reshape(1, X.shape[0], X.shape[1], 1)
Y = Y.reshape(1, Y.shape[0], Y.shape[1], 2)

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(None, None, 1)))
model.add(Conv2D(8, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', strides=2))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3,3), activation='tanh', padding='same'))

# Finish model
model.compile(optimizer='rmsprop', loss='mse')
model.fit(x=X, y=Y, batch_size=1, epochs=1000, verbose=0)

In [ ]:
model.evaluate(X, Y, batch_size=1)

In [ ]:
output = model.predict(X)
cur = np.zeros((200, 200, 3))
cur[:,:,0] = X[0][:,:,0]
cur[:,:,1:] = output[0]


#imshow(rgb_image.astype('float32'))

#cur = (cur * [100, 255, 255]) - [0 ,128, 128]
#output

#rgb_image *= [100, 255, 255]
cur = (cur * [100, 255, 255]) - [0, 128, 128]
rgb_image = lab2rgb(cur)
imshow(rgb_image)

# References
https://github.com/microic/niy/tree/master/examples/colorizing_photos